In [2]:
import requests
import bs4
import pymysql.cursors
from tabulate import tabulate



#create a database connection

import pymysql.cursors

# wrap Db in class
class DB_Query_Main():
    
    #connection to db
    def connect_to_db(self):
        connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='techbridge',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
        return connection

    
   
    #create a table
    def table(self, command):
        connection = self.connect_to_db()
        
        try:
            with connection.cursor() as cursor:
                cursor.execute(command)
            
            connection.commit()
        
        finally:
            connection.close()
            
        
    #fetch from the db
    def fetch_db(self, command, qty_to_fetch = 10):
        connection = self.connect_to_db()
        
        try:
            with connection.cursor() as cursor:
                cursor.execute(command)
                
                if qty_to_fetch == "all":
                    result = cursor.fetchall()
                else:
                    result = cursor.fetchmany(qty_to_fetch)
        finally:
            connection.close()
        return result
        
    #write to Db
    def write_db(self, command):
        
        connection = self.connect_to_db()
        
        try:
            with connection.cursor() as cursor:
                cursor.execute(command)
                
            connection.commit()
        
        finally:
            connection.close()
        




   

In [3]:
url_source = "https://www.jumia.com.ng/computing/hp/?q=hp+laptops"
url_data = requests.get(url_source)

html_content = bs4.BeautifulSoup(url_data.content)

all_items = html_content("article", {"class":"prd"})

products = []
prices = []



#all_products_prices.write("products,prices\n")

for item in all_items:
    product_names = item.find("h3", {"class":"name"})
    product_prices = item.find("div", {"class":"prc"})
    
#     print(product_prices.text)   
    
    try:
        products.append(product_names.text.
                       replace(";", "").
                       replace(",", "")) 
        prices.append(int(product_prices.text.replace("₦", "").replace(",", "")))
                
    except:
        continue
file = open("Products_and_Prices.csv", "w") 
file.write("product Names, Product Prices\n")
for product, price in zip(products, prices):
    product_prices = (price)
    product_names = (product)
    
     file.write(f"{product_names}, {product_prices}\n")

        
        
file.close()
print(products)
print(prices)
    

In [4]:

print("Minimum Price = " , min(prices))
print("\nMaximum Price = " ,max(prices))


Minimum Price =  119000

Maximum Price =  650000


In [5]:
all_products = list(zip(products, prices))

max_product = sorted(all_products, key = lambda products : products[1], reverse = True)[0]
min_product = sorted(all_products, key = lambda products : products[1], reverse = True)[-1]

print(max_product)
print(min_product)

('Hp ENVY 15 X360 CORE I7 16GB 1TB SSD 15.6" IPS 4GB NVIDIA MX330', 650000)
('Hp Stream 11 Intel Celeron 2GB RAM- 32GB HDD WIN 10+ 32GB FLASH', 119000)


In [7]:
messenger = DB_Query_Main()

#create table 

messenger.table("CREATE TABLE IF NOT EXISTS jumiaProducts(id int(5) NOT NULL AUTO_INCREMENT PRIMARY KEY, product_name varchar(55), product_price int(55))")

for product, price in zip(products, prices):
    messenger.write_db(f"INSERT INTO jumiaProducts(product_name, product_price) VALUES('{product}','{price}')")

In [11]:
# raw_data = "https://www.aliexpress.com/af/iPhones.html?d=y&origin=n&SearchText=iPhones&catId=0&initiative_id=SB_20210116063930"
# url_data = requests.get(raw_data)

# html_content = bs4.BeautifulSoup(url_data.content)

# print(html_content)


In [10]:
# product_list = html_content.find_all("li", {"class": "list-item"})
# product_site = []
# for items in product_list:
#     product_name = items.find("li", {"class":"list-item"})
    
#     product_site.append(product_name.text)
    
# print(product_list)